# Part 2 Create a DBaas (Database as a service)  POSTGRES IN EC2

### Note : Please Make sure to start the Postgres SQL DB Server , before connection of DBaaS 

### Import of Packages 

In [1]:
# !pip install psycopg2
# !pip install boto3

import boto3
import psycopg2
import pandas as pd
print('Import Done')

Import Done


### Connecting to Ec2 instance

In [2]:
ec2 = boto3.resource('ec2')
for instance in ec2.instances.all():
    print instance.id, instance.state

print('Conection to ec2 Sucessfull')

i-0fc9cd67f33ef81bc {u'Code': 16, u'Name': 'running'}
Conection to ec2 Sucessfull


### Defining the Public DNS of ec2 instance

In [19]:
hostaws = 'ec2-54-164-174-112.compute-1.amazonaws.com'

### Connecting PostgresSQL Host in EC2

In [20]:
conn = psycopg2.connect(host=hostaws,user='postgres',database='postgres')
print ("Opening connection using DSN:"+str(hostaws))
print "Encoding for this connection is", conn.encoding

Opening connection using DSN:ec2-54-164-174-112.compute-1.amazonaws.com
Encoding for this connection is UTF8


### Creating Table in Postgres

In [37]:
curs = conn.cursor()
try:
    curs.execute("CREATE TABLE properties (index int,parcelid int,bathroomcnt double precision,bedroomcnt double precision, buildingqualitytypeid double precision,calculatedfinishedsquarefeet double precision,fips double precision,heatingorsystemtypeid double precision,latitude double precision,longitude double precision,lotsizesquarefeet double precision,rawcensustractandblock  double precision, regionidcity double precision, regionidcounty double precision, roomcnt double precision,yearbuilt  double precision,structuretaxvaluedollarcnt  double precision,taxamount  double precision,censustractandblock  double precision,transactiondate date)")
except:
    conn.rollback()
    curs.execute("DROP TABLE properties")
    curs.execute("CREATE TABLE properties (index int,parcelid int,bathroomcnt double precision,bedroomcnt double precision, buildingqualitytypeid double precision,calculatedfinishedsquarefeet double precision,fips double precision,heatingorsystemtypeid double precision,latitude double precision,longitude double precision,lotsizesquarefeet double precision,rawcensustractandblock  double precision, regionidcity double precision, regionidcounty double precision, roomcnt double precision,yearbuilt  double precision,structuretaxvaluedollarcnt  double precision,taxamount  double precision,censustractandblock  double precision,transactiondate date)")
    curs.close()

### Removing the first column of the clean csv file , so that we can easily import the data into the Table

In [38]:
with open('cleanData.csv', 'r') as fin:
    data = fin.read().splitlines(True)
with open('texter.csv', 'w') as fout:
      fout.writelines(data[1:])

### Opening the fitered data 

In [39]:
f = open("texter.csv",'r')

###  Exporting the the local datafile to postgressql residing in the EC2 instance

In [40]:
curs = conn.cursor()
curs.copy_from(f, 'properties', sep=",")
conn.commit()

### Getting back the data formthe PostSQL DB server to local Dataframe

In [41]:
curs = conn.cursor()
curs.execute('SELECT * FROM properties;')
records = curs.fetchall()

### Converting to Dataframe to further analysis as required 

In [42]:
dataf = pd.DataFrame(records)

### Renaming the columns as per fetched , which will help ahead for querying purpose 

In [43]:
dataf.columns=['index','parcelid','longitude',
              'latitude','landtaxvaluedollarcnt','lotsizesquarefeet',
              'unitcnt','bathroomcnt','bedroomcnt','calculatedfinishedsquarefeet',
              'fips','fullbathcnt','heatingorsystemtypeid','rawcensustractandblock','yearbuilt',
              'structuretaxvaluedollarcnt','taxamount','censustractandblock','logerror','transactiondate']

### Sample Data fetchefrom the EC2 instance (Postgres SQL DB server )

In [44]:
dataf.head()

,index,parcelid,longitude,latitude,landtaxvaluedollarcnt,lotsizesquarefeet,unitcnt,bathroomcnt,bedroomcnt,calculatedfinishedsquarefeet,fips,fullbathcnt,heatingorsystemtypeid,rawcensustractandblock,yearbuilt,structuretaxvaluedollarcnt,taxamount,censustractandblock,logerror,transactiondate
0,1285,10726315,3.0,3.0,4.0,2445.0,6037.0,2.0,34184300.0,-118657000.0,63878.0,6.037135e+07,12447.0,3101.0,0.0,1982.0,436551.0,7170.22,6.037135e+13,2016-08-24
1,1309,10730788,2.0,4.0,7.0,1570.0,6037.0,2.0,34188446.0,-118616724.0,11308.0,6.037135e+07,12447.0,3101.0,0.0,1959.0,115379.0,5097.78,6.037135e+13,2016-07-01
2,1361,10743512,3.0,2.0,4.0,1394.0,6037.0,2.0,34152890.0,-118791494.0,77543.0,6.037800e+07,34278.0,3101.0,0.0,1998.0,203426.0,5550.36,6.037800e+13,2016-03-31
3,1417,10760933,2.0,4.0,7.0,1212.0,6037.0,7.0,34220631.0,-118521001.0,7379.0,6.037131e+07,12447.0,3101.0,0.0,1954.0,32615.0,4560.02,6.037131e+13,2016-06-27
4,1456,10772282,3.0,4.0,4.0,2597.0,6037.0,2.0,34209413.0,-118578052.0,10537.0,6.037134e+07,12447.0,3101.0,0.0,1964.0,234987.0,4998.86,6.037134e+13,2016-04-15


###### The END 